# Pré-processamento: Parte II

## Importar os pacotes que serão usados

In [ ]:
import os
import numpy as np
from osgeo import gdal
from osgeo.gdalconst import GA_ReadOnly
from multiprocessing import Pool, Manager, cpu_count
from osgeo_utils.gdal_calc import *
from concurrent.futures import ProcessPoolExecutor

## Criar o retângulo envolvente

Por questões de otimização e para que o código a seguir pudesse ser paralelizado a fim de tornar o processo mais rápido e eficiente, o código abaixo recorta do rasterd do Brasil do MapBiomas o menor retângulo possível que envolve cada bioma.
O tamanho desse retângulo é dado pela extensão das máscaras de bioma criadas a partir dos rasters do inventário no processo anterior.

In [ ]:
def process_file(arquivo):
    print(f"Process for {arquivo} started...\n")
    # ler o arquivo que sera usado para pegar a extensao do recorte
    read_file = rf"{pasta}/{arquivo}"
    Image = gdal.Open(read_file)

    # aqui será extraído um recorte da seguinte extensão do SRC:
    gt = Image.GetGeoTransform()

    ulx = gt[0]
    uly = gt[3]
    lrx = ulx + gt[1] * Image.RasterXSize
    lry = uly + gt[5] * Image.RasterYSize

    # texto padronizado para definir o sistema de projeção:
    gp = Image.GetProjection()

    # abrir camada de entrada que sera recortada
    src_f = '../dados/mapbiomas_v7_2016/mapa_brasil/mapbiomas_v7_sirgas2000.tif'
    src_ds = gdal.Open(src_f, 0)  # carrega a camada raster em um objeto "dataset"
 
    # definir o nome da camada de saída
    out_f = f"../dados/mapbiomas_v7_2016/bounding_box/bb{arquivo.split('inventario')[1]}"  # incluir a extensão de saída

    # definição das opções em um objeto de opções
    options = gdal.TranslateOptions(format='GTiff', bandList=[1], projWin=[ulx, uly, lrx, lry], projWinSRS=gp, noData=0, outputType=gdal.GDT_Byte)  
    
    # convocar a função Translate e passar o objeto opts no parâmetro 'options'
    out_ds = gdal.Translate(destName=out_f, srcDS=src_ds, options=options)
    
    # fechar os datasets:
    src_ds = None
    out_ds = None
    Image = None
    
    return arquivo

def handle_result(result):
    print(f"Process for {result} has finished.")

if __name__ == '__main__':
    # Especifica o caminho da pasta que contém os arquivos
    pasta = '../dados/inventario_v4_2016/inventario_biomas_rasters'

    # Obtém a lista de arquivos na pasta
    arquivos = os.listdir(pasta)

    # Cria uma pool de processos
    with Pool(processes=3) as pool, Manager() as manager:
        # Cria um dicionário para armazenar os resultados
        results_dict = manager.dict()

        # Submete tarefas assíncronas para cada arquivo na pasta
        results = [pool.apply_async(process_file, args=(arquivo,), callback=handle_result) for arquivo in arquivos]

        # Espera todas as tarefas completarem
        for result in results:
            result.wait()

        # Obtém os resultados
        for result in results:
            results_dict[result.get()] = True

        # Verifica se todos os processos foram concluídos
        while len(results_dict) != len(arquivos):
            pass

        print("All processes have finished.")

        # Aguarda todos os processos terminarem
        pool.close()
        pool.join()

## Recortar o retângulo envolvente usando a máscara de biomas

A partir dos retângulo envolvente de cada bioma e da máscara de

In [ ]:
# Supress the warning
np.seterr(divide='ignore', invalid='ignore')

# Função que processa um arquivo
def process_file(arquivo, pasta_biomas_mask, n_processo, status):
    print(f"Processo {n_processo}: processando o arquivo {arquivo}...\n")
    A=f"../dados/biomas_mask/{arquivo}"
    B=f"../dados/mapbiomas_v7_2016/bounding_box/bb{arquivo.split('mask')[1]}"
    outfile=f"../dados/mapbiomas_v7_2016/biomas/mapbiomas{arquivo.split('mask')[1]}"
    calc="(A>0)*B"


    gdal_call = f'gdal_calc.py --calc="{calc}" --outfile={outfile} -A {A} -B {B} --type=Byte --overwrite --quiet'
    print(gdal_call)
    os.system(gdal_call)

    
    print(f"Processo {n_processo}: o arquivo {arquivo} foi processado com sucesso!\n")
    status[arquivo] = False


if __name__ == '__main__':
    # Especifica o caminho da pasta que contém os arquivos
    pasta_biomas_mask = '../dados/biomas_mask'

    # Obtém a lista de arquivos na pasta
    arquivos = os.listdir(pasta_biomas_mask)

    # Define o número máximo de processos em execução simultaneamente
    max_processes = 3

    # Inicializa o pool de processos e o objeto Manager para compartilhar dados entre processos
    pool = Pool(processes=max_processes)
    manager = Manager()

    # Cria um dicionário compartilhado para armazenar o status de processamento de cada arquivo
    status = manager.dict([(arquivo, False) for arquivo in arquivos])
    print(status)

    # Loop pelos arquivos e executa o mesmo processo em cada um, usando o pool de processos
    for i, arquivo in enumerate(arquivos):
       
        # Aguarda até que haja um processo livre para executar o próximo arquivo
        while True:
            if sum(status.values()) < max_processes:
                break

        # Define o status do arquivo como "em processamento"
        status[arquivo] = True

        # Processa o arquivo em um novo processo
        pool.apply_async(process_file, args=(arquivo, pasta_biomas_mask, i, status))

    # Aguarda todos os processos terminarem
    pool.close()
    pool.join()

    # Verifica se todos os processos foram finalizados
    if sum(status.values()) == 0:
        print("Todos os arquivos foram processados com sucesso!")

## Gerar os mapas de UCT por bioma

Usar o retângulo envolvente e uma máscara binária para selecionar a região que será comparada

In [ ]:
# Supress the warning
np.seterr(divide='ignore', invalid='ignore')

# Especifica o caminho da pasta que contém os arquivos
pasta_biomas_mask = '../dados/biomas_mask'

# Obtém a lista de arquivos na pasta
arquivos = os.listdir(pasta_biomas_mask)

for arquivo in arquivos:

    print(f"Processo: processando o arquivo {arquivo}...\n")
    A=f"../dados/biomas_mask/{arquivo}"
    B=f"../dados/mapbiomas_v7_2016/bounding_box/bb{arquivo.split('mask')[1]}"
    outfile=f"../dados/mapbiomas_v7_2016/biomas/mapbiomas{arquivo.split('mask')[1]}"

    Calc("(A > 0) * B ", 
            A=A, 
            B=B, 
            outfile=outfile,
            type='Byte', overwrite=True, extent='intersect', quiet=True)
    
    print(f"Processo: o arquivo {arquivo} foi processado com sucesso!\n")
    
print("Todos os arquivos foram processados com sucesso!")